In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers

tf.enable_eager_execution()
pd.set_option('float_format', '{:f}'.format)
tf.__version__

'1.13.1'

In [5]:
%%time
df_meta = pd.read_csv('item_metadata.csv', index_col='item_id')
df_train = pd.read_csv('train.csv')

Wall time: 42.1 s


In [6]:
%%time
max_meta_len = df_meta['properties'].apply(lambda prop: len(prop.split('|'))).max()
max_meta_len

Wall time: 1.41 s


In [7]:
class FastLabelEncoder:
    def __init__(self):
        self.token_to_id = dict()
        self.id_to_token = dict()
        self.unk_id = 1
        self.pad_id = 0
        self.current_id = 2

    def __len__(self):
        return len(self.token_to_id)

    def _pad(self, seq, pad):
        if pad < 0: return seq
        if len(seq) > pad: return seq[:pad]
        return seq + [self.pad_id] * (pad - len(seq))

    def fit(self, X):
        for row in X:
            for token in row:
                if token not in self.token_to_id:
                    self.token_to_id[token] = self.current_id
                    self.id_to_token[self.current_id] = token
                    self.current_id += 1

        return self

    def transform(self, X, pad=-1):
        return np.array([self._pad([
              self.token_to_id[token] if token in self.token_to_id else self.unk_id
              for token in row
        ], pad) for row in X])
        
    def fit_transform(self, X, pad=-1):
        return self.fit(X).transform(X, pad)

In [35]:
class Input:
    col_shapes = {
        'type': tf.TensorShape([None]),
        'value': tf.TensorShape([None, max_meta_len]),
        'neg': tf.TensorShape([None]),
        'pos': tf.TensorShape([])
    }
    out_shapes = tuple(v for k, v in col_shapes.items())

    def __init__(self):
        self.action_types_encoder = FastLabelEncoder()
        self.action_vals_encoder = FastLabelEncoder()
        self.labels_encoder = FastLabelEncoder()

    def generator(self):
        last_session_id = None
        action_types = []
        action_vals = []
        MISSING_REF_VAL = '?'

        for _, row in df_train.iterrows():
            if row['session_id'] != last_session_id:
                action_types = []
                action_vals = []

            action_val = row['reference']

            if row['action_type'] != 'clickout item':
                action_types.append(row['action_type'])

                if action_val.isdigit():
                    int_id = int(action_val)
                    if int_id in df_meta.index:
                        action_vals.append(df_meta.loc[int_id]['properties'].split('|'))
                    else:
                        action_vals.append([int_id])
                else:
                    action_vals.append([action_val])
            else:
                neg = [int(i) for i in row['impressions'].split('|') if i != action_val]
                yield self.action_types_encoder.fit_transform([action_types])[0], \
                      self.action_vals_encoder.fit_transform(action_vals, pad=max_meta_len), \
                      self.labels_encoder.fit_transform([neg])[0], \
                      self.labels_encoder.fit_transform([[int(action_val)]])[0][0]

            last_session_id = row['session_id']

    def as_dataset(self, batch_size):
        dataset = tf.data.Dataset.from_generator(
            self.generator,
            (tf.int32,) * 4,
            self.out_shapes
        )

        dataset = dataset.padded_batch(batch_size, padded_shapes=self.out_shapes)

        return dataset

In [36]:
next(Input().generator())

(array([2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
 array([[ 2,  0,  0, ...,  0,  0,  0],
        [ 3,  4,  5, ...,  0,  0,  0],
        [ 3,  4,  5, ...,  0,  0,  0],
        ...,
        [28, 29, 30, ...,  0,  0,  0],
        [28, 29, 30, ...,  0,  0,  0],
        [28, 29, 30, ...,  0,  0,  0]]),
 array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25]),
 26)

In [10]:
%%time
i = Input().generator()
next(i)
for _ in range(5000):
    next(i)

Wall time: 13.8 s


too slow 

https://stackoverflow.com/questions/24870953/does-pandas-iterrows-have-performance-issues

In [37]:
inp = Input()
dataset = inp.as_dataset(batch_size=3)

In [38]:
dataset.make_one_shot_iterator().get_next()

(<tf.Tensor: id=4017, shape=(3, 34), dtype=int32, numpy=
 array([[2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4]])>,
 <tf.Tensor: id=4018, shape=(3, 34, 112), dtype=int32, numpy=
 array([[[ 2,  0,  0, ...,  0,  0,  0],
         [ 3,  4,  5, ...,  0,  0,  0],
         [ 3,  4,  5, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
        [[ 2,  0,  0, ...,  0,  0,  0],
         [ 3,  4,  5, ...,  0,  0,  0],
         [ 3,  4,  5, ...,  0,  0,  0],
         ...,
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0],
         [ 0,  0,  0, ...,  0,  0,  0]],
 
 

---

In [39]:
action_type_emb_size = 20
action_val_emb_size = 20
output_emb_size = 20

action_type = layers.Embedding(
    input_dim=len(inp.action_types_encoder),
    output_dim=action_type_emb_size,
    mask_zero=True
)(layers.Input(shape=Input.col_shapes['type']))

action_val = layers.Embedding(
    input_dim=len(inp.action_vals_encoder),
    output_dim=action_val_emb_size,
    mask_zero=True
)(layers.Input(shape=Input.col_shapes['value']))

pos = layers.Embedding(
    input_dim=len(inp.labels_encoder),
    output_dim=output_emb_size,
    mask_zero=True
)(layers.Input(shape=Input.col_shapes['pos']))

neg = layers.Embedding(
    input_dim=len(inp.labels_encoder),
    output_dim=output_emb_size,
    mask_zero=True
)(layers.Input(shape=Input.col_shapes['neg']))

dimension_of_seq_features = 2
action_val_reduced = tf.reduce_mean(action_val, dimension_of_seq_features)

seq = layers.Concatenate(axis=2)([action_type, action_val_reduced])

In [40]:
# (batch, sequence, embedding of items in sequence)
seq, pos, neg

(<tf.Tensor 'concat_12:0' shape=(?, ?, 40) dtype=float32>,
 <tf.Tensor 'embedding_lookup_22/Identity_2:0' shape=(?, 20) dtype=float32>,
 <tf.Tensor 'embedding_lookup_23/Identity_2:0' shape=(?, ?, 20) dtype=float32>)

In [52]:
import SASRec.modules as sas_module

In [55]:
num_blocks = 2
hidden_units = 40
num_heads = 1
dropout_rate = 0.5
is_training = True
max_seq_len = 500

mask = tf.to_float(tf.not_equal(seq, 0))

for i in range(num_blocks):
    with tf.variable_scope('num_blocks_%d' % i):
        # self-attention
#         seq = sas_module.multihead_attention(
#             queries=sas_module.normalize(seq),
#             keys=seq,
#             num_units=hidden_units,
#             num_heads=num_heads,
#             dropout_rate=dropout_rate,
#             is_training=is_training,
#             causality=True,
#             scope='self_attention'
#         )

        # Feed forward
        seq = sas_module.feedforward(
            sas_module.normalize(seq),
            num_units=[hidden_units, hidden_units],
            dropout_rate=dropout_rate,
            is_training=is_training
        )

        seq *= mask

seq = sas_module.normalize(seq)

# self.pos_logits = tf.reduce_sum(pos_emb * seq, -1)
# self.neg_logits = tf.reduce_sum(neg_emb * seq, -1)

In [30]:
model = layers.Bidirectional(
    layers.LSTM(128, dropout=0.25, return_sequences=False, recurrent_dropout=0.1)
)(seq)

model = layers.Dense(units=1)(model)

In [29]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
from IPython.display import Image
Image(retina=True, filename='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.